In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
sns.set_style("white")

In [ ]:
# Display figures at a reasonable default size.
mpl.rcParams['figure.figsize'] = (6, 4)

# Disable top and right spines.
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
    
# Display and save figures at higher resolution for presentations and manuscripts.
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['figure.dpi'] = 300

# Display text at sizes large enough for presentations and manuscripts.
mpl.rcParams['font.weight'] = "normal"
mpl.rcParams['axes.labelweight'] = "normal"
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

## Load data

Load exhaustive grid search data. For each possible embedding method, corresponding method parameters, and HDBSCAN distance threshold, we produced an embedding for training and validation data (using 2-fold validation with 3 repeats for N=6 cross-validation iterations per parameter combination), assigned clusters to each embedding, and evaluated how well all pairs of strains in the data were assigned to the same or different cluster compared to predetermined clade assignments.

In [ ]:
grid = pd.read_csv(snakemake.input.table)

In [ ]:
grid.head()

In [ ]:
list(grid.columns)

## Identify optimal method parameter values

Find the method parameters for each method that maximizes the $R^{2}$ value across all replicates.

In [ ]:
grid_columns = [
    "method",
    "components",
    "perplexity",
    "learning_rate",
    "nearest_neighbors",
    "min_dist",
]

In [ ]:
grid

In [ ]:
grid.groupby(grid_columns, dropna=False)["r_squared"].agg(["mean", "std"]).reset_index()

### PCA

In [ ]:
pca_grid = grid.query("method == 'pca'")

In [ ]:
pca_grid.shape

In [ ]:
pca_grid

In [ ]:
pca_mean_r_squared = pca_grid.groupby([
    "method",
])["r_squared"].mean().reset_index().sort_values(
    "r_squared",
    ascending=False
)

In [ ]:
pca_mean_r_squared

In [ ]:
pca_max_r_squared = pca_mean_r_squared.sort_values("r_squared", ascending=False).head(1)

In [ ]:
pca_max_r_squared

In [ ]:
pca_max_r_squared.to_csv(
    snakemake.output.pca_parameters,
    index=False,
)

### MDS

In [ ]:
mds_grid = grid.query("method == 'mds'")

In [ ]:
mds_grid.shape

In [ ]:
mds_mean_r_squared = mds_grid.groupby([
    "method",
    "components",
])["r_squared"].mean().reset_index().sort_values(
    "r_squared",
    ascending=False
)

In [ ]:
mds_mean_r_squared

In [ ]:
mds_max_r_squared = mds_mean_r_squared.sort_values("r_squared", ascending=False).head(1)

In [ ]:
mds_max_r_squared.to_csv(
    snakemake.output.mds_parameters,
    index=False,
)

In [ ]:
facet_grid = sns.catplot(
    data=mds_grid,
    x="components",
    y="r_squared",
    dodge=True,
    alpha=0.75,
)

for ax in facet_grid.axes.flatten():
    ax.set_ylabel("$R^{2}$ of Euclidean and genetic distance")
    ax.set_ylim(0, 1)

plt.savefig(snakemake.output.score_by_mds_parameters)

### t-SNE

In [ ]:
tsne_grid = grid.query("method == 't-sne'")

In [ ]:
tsne_grid.shape

In [ ]:
tsne_mean_r_squared = tsne_grid.groupby([
    "method",
    "perplexity",
    "learning_rate"
])["r_squared"].mean().reset_index().sort_values(
    "r_squared",
    ascending=False
)

In [ ]:
tsne_mean_r_squared

In [ ]:
tsne_max_r_squared = tsne_mean_r_squared.sort_values("r_squared", ascending=False).head(1)

In [ ]:
tsne_max_r_squared

In [ ]:
tsne_max_r_squared.to_csv(
    snakemake.output.tsne_parameters,
    index=False,
)

In [ ]:
facet_grid = sns.catplot(
    data=tsne_grid,
    x="perplexity",
    y="r_squared",
    hue="learning_rate",
    dodge=True,
    alpha=0.75,
)

for ax in facet_grid.axes.flatten():
    ax.set_ylabel("$R^{2}$ of Euclidean and genetic distance")
    ax.set_ylim(0, 1)

plt.savefig(snakemake.output.score_by_tsne_parameters)

### UMAP

In [ ]:
umap_grid = grid.query("method == 'umap'")

In [ ]:
umap_grid.head()

In [ ]:
umap_grid.shape

In [ ]:
umap_mean_r_squared = umap_grid.groupby([
    "method",
    "min_dist",
    "nearest_neighbors",
])["r_squared"].mean().reset_index().sort_values(
    "r_squared",
    ascending=False
)

In [ ]:
umap_mean_r_squared

In [ ]:
umap_max_r_squared = umap_mean_r_squared.sort_values("r_squared", ascending=False).head(1)

In [ ]:
umap_max_r_squared

In [ ]:
umap_max_r_squared.to_csv(
    snakemake.output.umap_parameters,
    index=False,
)

In [ ]:
facet_grid = sns.catplot(
    data=umap_grid,
    x="min_dist",
    y="r_squared",
    hue="nearest_neighbors",
    dodge=True,
    alpha=0.75,
)

for ax in facet_grid.axes.flatten():
    ax.set_ylabel("$R^{2}$ of Euclidean and genetic distance")
    ax.set_ylim(0, 1)

plt.savefig(snakemake.output.score_by_umap_parameters)

## Find best correlation per method

In [ ]:
max_r_squared = pd.concat([pca_max_r_squared, mds_max_r_squared, tsne_max_r_squared, umap_max_r_squared])

In [ ]:
max_r_squared

In [ ]:
grid.head()

In [ ]:
grid_summary = grid.merge(
    max_r_squared,
    on=grid_columns,
    suffixes=["", "_mean"]
).groupby([
    "method",
    "recombination_rate"
]).aggregate({
    "replicate": ["count"],
    "r_squared": ["mean", "std"],
    "slope": ["mean", "std"],
    "intercept": ["mean", "std"],
})

In [ ]:
grid_summary

In [ ]:
grid_summary.columns

In [ ]:
new_columns = ["_".join(columns) for columns in grid_summary.columns]

In [ ]:
grid_summary = grid_summary.set_axis(new_columns, axis=1).reset_index()

In [ ]:
grid_summary

In [ ]:
grid_summary.to_csv(
    snakemake.output.summary_score_by_method,
    index=False,
)